In [4]:
# pip install mysql-connector-python


  Using cached mysql_connector_python-8.1.0-cp310-cp310-manylinux_2_17_x86_64.whl (27.5 MB)
  Using cached protobuf-4.21.12-cp37-abi3-manylinux2014_x86_64.whl (409 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.154.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.21.12 which is incompatible.
sagemaker-training 4.5.0 requires protobuf<=3.20.3,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
smdebug 1.0.34 requires protobuf<=3.20.3,>=3.20.0, but you have protobuf 4.21.12 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages

In [4]:
# Imported Library
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import mysql.connector

In [5]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [6]:
# Importing data from csv
# %store -r df
# print(df)  # This will print "Hello, world!"

# Read CSV file into a DataFrame
df = pd.read_csv('virus-slow50.csv')

# data = pd.read_csv("subset2.csv")  
# data.tail()
print(df)

     0  1  2  3  4  5  6  7  8  9  ...  4023  4024  4025  4026  4027  4028   
0    0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0  \
1    0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2    0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
3    0  0  0  0  0  0  0  0  0  0  ...     1     1     1     1     1     1   
4    0  0  0  0  0  0  0  0  0  0  ...     1     1     1     1     1     1   
..  .. .. .. .. .. .. .. .. .. ..  ...   ...   ...   ...   ...   ...   ...   
463  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
464  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
465  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
466  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
467  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   

     4029  4030  4031  4032  
0       0     0     0     0  
1  

In [7]:
# Database connection parameters
db_config = {
    'host': 'proxx-rtls-cv-prototype.cxg1ksdtmfbm.eu-west-2.rds.amazonaws.com',
    'user': 'riv',
    'password': 'Call me Ishmae1,',
    'database': 'disease_transmission_simulation',
    'port': 3306
}

In [8]:
# Connect to the database
conn = mysql.connector.connect(**db_config)

# Check if the connection is open
if conn.is_connected():
    print("Connection is open.")
else:
    print("Connection is closed.")

Connection is open.


In [9]:
person_ids = np.arange(0, 468)
# print(person_ids)
# len(person_ids)

In [10]:
import torch
from torch_geometric.data import Data, TemporalData

# Create an empty list to store graph data for all timesteps
graph_data_list = []

# Loop over all timesteps (from t=0 to t=19)
# 2016
for timestep in range(4032):
    
    # data_t = data[data['timestep'] == timestep]
    # labels_t = unique_prob[unique_prob['timestep'] == timestep]
    
    query = f"""
    SELECT *
    FROM distinct_records_sorted
    WHERE timestep = {timestep}
    """

    # Fetch data into a Pandas DataFrame
    current_snapshot = pd.read_sql(query, conn)
    # Replace 468 with 0 in 'min_person_id' column
    current_snapshot['min_person_id'] = current_snapshot['min_person_id'].replace(468, 0)
    # Replace 468 with 0 in 'max_person_id' column
    current_snapshot['max_person_id'] = current_snapshot['max_person_id'].replace(468, 0)

    
    # Extract the list for t=0
    current_labels =  df.iloc[:, timestep].tolist()
    
    query = f"""
    SELECT *
    FROM distinct_records_sorted
    WHERE timestep = {timestep+1}
    """

    # Fetch data into a Pandas DataFrame
    subsequent_snapshot = pd.read_sql(query, conn)
    
    # Extract the list for t=0
    subsequent_labels =  df.iloc[:, timestep+1].tolist()
    
    # Example graph data for timestep t
    # Replace the following lines with your actual data for each timestep
    
    # num_nodes = 30  # 29+1
    # node_features_t = torch.tensor(current_labels)
    node_features_t = torch.tensor(current_labels).unsqueeze(1)

    
    edge_index_t = torch.tensor(current_snapshot[["min_person_id", "max_person_id"]].values, dtype=torch.long)  # Edge indices at timestep t
    edge_index_t = edge_index_t.transpose(0, 1)
    edge_attr_t = torch.tensor(current_snapshot["distance"].values)  # Edge attributes at timestep t
    
    # y_t = torch.tensor(subsequent_labels)  # Target labels (optional) at timestep t
    y_t = torch.tensor(subsequent_labels)


    # Create graph data for timestep t
    graph_data_t = Data(x=node_features_t, edge_index=edge_index_t, edge_attr=edge_attr_t, y=y_t)

    # Add the graph data to the list
    graph_data_list.append(graph_data_t)

print(graph_data_list[0])

# Now you have the temporal graph dataset containing graph data for all 20 timesteps
# You can use this dataset as input to PyTorch Geometric GNN models for temporal graph data.
# For example, you can use this dataset to train a temporal GNN model to predict node labels at the subsequent timestep.


Data(x=[468, 1], edge_index=[2, 46971], edge_attr=[46971], y=[468])


In [11]:
import pickle

# Save data to a pickle file
with open('graph-slow50.pkl', 'wb') as f:
    pickle.dump(graph_data_list, f)